# **Fine-tuning XLS-R for Guarani**

We used this blog post as reference:https://huggingface.co/blog/fine-tune-xlsr-wav2vec2.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov  2 14:52:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install datasets
#==1.18.3
!pip install transformers
#==4.11.3
!pip install torchaudio
#==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer
!pip install accelerate -U
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 45.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.7 MB/s eta 0:00:00
     ━━

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


## Prepare Data, Tokenizer, Feature Extractor

### Create `Wav2Vec2CTCTokenizer`

In [ ]:
from datasets import load_dataset, load_metric, Audio, concatenate_datasets, interleave_datasets

common_voice_train_gn = load_dataset("mozilla-foundation/common_voice_13_0", "gn", split="train+validation",use_auth_token=True)
common_voice_test_gn = load_dataset("mozilla-foundation/common_voice_13_0", "gn", split="test",use_auth_token=True)
common_voice_train_it = load_dataset("mozilla-foundation/common_voice_13_0", "it", split="train",use_auth_token=True)
common_voice_test_it = load_dataset("mozilla-foundation/common_voice_13_0", "it", split="test",use_auth_token=True)

common_voice_train = concatenate_datasets([common_voice_train_gn, common_voice_train_it]).shuffle(seed=42)
common_voice_test = common_voice_test_gn#interleave_datasets([common_voice_test_gn, common_voice_test_it], seed=42)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2089: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=True' instead.
  warnings.warn(


In [ ]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes","variant"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes","variant"])

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence
0,"Lui è un avvocato affermato, lei una psicologa di successo."
1,"Anche la canzone ""Squat That Rabbit"" è una cover, del cantante blues Taj Mahal."
2,Si è poi esteso alla quasi totalità del continente.
3,"Inoltre era anche la voce narrante, prendendo il posto di Zach Braff."
4,"Generalmente lavora nei generi drammatici, d'azione e horror."
5,"Nella frazione ""Busa"" si trova infine la chiesa della Ss."
6,Poco più a monte della cascata si trova invece l'alpeggio Vettismorki.
7,La Vergine era quindi venerata come la vera guida verso Cristo.
8,La penisola offre molte attrazioni di interesse archeologico.
9,In epoca successiva sono collocati gli orologi.


In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\…\„\’\–\(\)\+]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Map:   0%|          | 0/164403 [00:00<?, ? examples/s]

Map:   0%|          | 0/811 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,sebbene questo giornale dovesse presto chiudere i suoi articoli gli dettero notorietà come scrittore
1,sembra accertato lutilizzo di tali acque termali anche in epoca preromana
2,la madre si chiamava matilde viale
3,i colori sono molto variabili in base alla sottospecie presa in considerazione
4,fu venduta in versione berlina e landaulet
5,il lavoro di sahagún è conosciuto grazie ad un manoscritto chiamato codice fiorentino
6,questa famiglia povera chiede la carità ai piedi della chiesa della madeleine
7,è la forma più comune di leucemia cronica delletà evolutiva
8,si trattò del primo volo svoltosi per posizionare dei satelliti artificiali del tipo commerciale
9,si possono osservare sui cardi intenti a nutrirsi


In [ ]:
def replace_hatted_characters(batch):
    batch["sentence"] = re.sub('[â]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[î]', 'i', batch["sentence"])
    batch["sentence"] = re.sub('[ô]', 'o', batch["sentence"])
    batch["sentence"] = re.sub('[û]', 'u', batch["sentence"])
    return batch

In [ ]:
#common_voice_train = common_voice_train.map(replace_hatted_characters)
#common_voice_test = common_voice_test.map(replace_hatted_characters)

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/164403 [00:00<?, ? examples/s]

Map:   0%|          | 0/811 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 '$': 1,
 '/': 2,
 '<': 3,
 '=': 4,
 '>': 5,
 '[': 6,
 ']': 7,
 '_': 8,
 '`': 9,
 'a': 10,
 'b': 11,
 'c': 12,
 'd': 13,
 'e': 14,
 'f': 15,
 'g': 16,
 'h': 17,
 'i': 18,
 'j': 19,
 'k': 20,
 'l': 21,
 'm': 22,
 'n': 23,
 'o': 24,
 'p': 25,
 'q': 26,
 'r': 27,
 's': 28,
 't': 29,
 'u': 30,
 'v': 31,
 'w': 32,
 'x': 33,
 'y': 34,
 'z': 35,
 '{': 36,
 '}': 37,
 '~': 38,
 '¡': 39,
 '«': 40,
 '°': 41,
 '´': 42,
 'µ': 43,
 'º': 44,
 '»': 45,
 'ß': 46,
 'à': 47,
 'á': 48,
 'â': 49,
 'ã': 50,
 'ä': 51,
 'å': 52,
 'æ': 53,
 'è': 54,
 'é': 55,
 'ê': 56,
 'ë': 57,
 'ì': 58,
 'í': 59,
 'î': 60,
 'ï': 61,
 'ð': 62,
 'ñ': 63,
 'ò': 64,
 'ó': 65,
 'ô': 66,
 'õ': 67,
 'ö': 68,
 'ø': 69,
 'ù': 70,
 'ú': 71,
 'û': 72,
 'ý': 73,
 'þ': 74,
 'ÿ': 75,
 'ā': 76,
 'ą': 77,
 'ć': 78,
 'č': 79,
 'đ': 80,
 'ė': 81,
 'ę': 82,
 'ě': 83,
 'ğ': 84,
 'ĩ': 85,
 'ī': 86,
 'ı': 87,
 'ľ': 88,
 'ł': 89,
 'ń': 90,
 'ň': 91,
 'ō': 92,
 'ő': 93,
 'œ': 94,
 'ř': 95,
 'ś': 96,
 'ş': 97,
 'š': 98,
 'ũ': 99,
 'ū': 100,

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

56

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
import transformers

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
repo_name = "wav2vec2-large-xls-r-300m-gn-pt-colab"

In [ ]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/aninditr/wav2vec2-large-xls-r-300m-gn-pt-colab/commit/01a093f93b5ae7a0aa8c97dd9a282a7559da452d', commit_message='Upload tokenizer', commit_description='', oid='01a093f93b5ae7a0aa8c97dd9a282a7559da452d', pr_url=None, pr_revision=None, pr_num=None)

### Create `Wav2Vec2FeatureExtractor`

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

### Preprocess Data



In [ ]:
common_voice_train[0]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/d7151dd4261beb2d1e1af3bc04935bf696ee03b932750da8992357991a109f64/pt_train_0/common_voice_pt_25269893.mp3'

In [ ]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/d7151dd4261beb2d1e1af3bc04935bf696ee03b932750da8992357991a109f64/pt_train_0/common_voice_pt_25269893.mp3',
 'array': array([ 2.66453526e-15,  1.02140518e-13,  1.30562228e-13, ...,
        -5.83755173e-06,  1.39061012e-05,  9.20801813e-06]),
 'sampling_rate': 48000}

In [ ]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/d7151dd4261beb2d1e1af3bc04935bf696ee03b932750da8992357991a109f64/pt_train_0/common_voice_pt_25269893.mp3',
 'array': array([-5.45696821e-12, -1.45519152e-11,  3.63797881e-11, ...,
         2.29360603e-06, -1.59090050e-05, -1.35091796e-05]),
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

mesmo nos dias mais frios os homens saem e brincam com seus aviões modelo


In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: você é um hipnotizador
Input array shape: (74112,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/21714 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/811 [00:00<?, ? examples/s]

In [ ]:
max_input_length_in_sec = 5.0
common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter:   0%|          | 0/21714 [00:00<?, ? examples/s]

## Training



### Set-up Trainer



In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1925: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
  report_to = 'wandb',
  run_name = 'custom_training'
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

### Training

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
400,6.309300,3.292210,1.000000
800,1.525500,1.336482,0.987790
1200,0.904500,1.038892,0.966647
1600,0.738100,0.869792,0.916617
2000,0.650400,0.798472,0.873734
2400,0.531500,0.969252,0.893389
2800,0.506800,0.968727,0.899345
3200,0.470900,0.812064,0.831149
3600,0.416600,0.790962,0.812388
4000,0.411900,0.793964,0.822513


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Buffered data was truncated after reaching the output size limit.Buffered data was truncated after reaching the output size limit.

In [ ]:
trainer.evaluate()

Step,Training Loss,Validation Loss,Wer
400,6.309300,3.292210,1.000000
800,1.525500,1.336482,0.987790
1200,0.904500,1.038892,0.966647
1600,0.738100,0.869792,0.916617
2000,0.650400,0.798472,0.873734
2400,0.531500,0.969252,0.893389
2800,0.506800,0.968727,0.899345
3200,0.470900,0.812064,0.831149
3600,0.416600,0.790962,0.812388
4000,0.411900,0.793964,0.822513


{'eval_loss': 0.9576923847198486, 'eval_wer': 0.7060750446694462}

In [ ]:
trainer.push_to_hub()

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

'https://huggingface.co/aninditr/wav2vec2-large-xls-r-300m-gn-pt-colab/tree/main/'